<h1>3. 1 Predicting the outcome of a loan application with Keras NN</h1>

<h3>3.1) Implement a Keras NN</h3>
<li>Preprocessing of data for Keras</li>
<li>Find the best parameters using Gridsearch</li>
<li>Select the best model and fine tune paremeters</li>
<li>Compare the two approaches</li>





In [1]:
import pandas as pd
import numpy as np

In [2]:
kiva_keras = pd.read_csv('data_files/kiva_loans.csv')

<h2>3.11 Preprocessing of data for Keras</h2>

In [3]:
#create the extra column
kiva_keras.insert(loc = 3, column='loan_status',value = 'placeholder')

In [4]:
#compute and insert non fondet loans
condition1 = (kiva_keras['funded_amount'] ==0)
kiva_keras.loc[condition1, 'loan_status'] = 'not funded'
not_funded = kiva_keras.loc[kiva_keras.loan_status == 'not funded']

condition2 = (kiva_keras['loan_amount'] - kiva_keras['funded_amount'].astype(int)>0) & (kiva_keras['funded_amount'] !=0)
kiva_keras.loc[condition2, 'loan_status'] = 'partially funded'
partially_funded = kiva_keras.loc[kiva_keras.loan_status == 'partially funded']

#compute and insert fully fondet loans
condition3 = (kiva_keras['loan_amount'] - kiva_keras['funded_amount'].astype(int)==0)
kiva_keras.loc[condition3, 'loan_status'] = 'Totally funded'
totally_funded = kiva_keras.loc[kiva_keras.loan_status == 'Totally funded']
#remove two overfundet loans
notvalid = (kiva_keras['loan_status']=='placeholder')
kiva_keras = kiva_keras.drop(kiva_keras[notvalid].index)


<b>preprocess the gender in order to get 4 classes: Women, Male, Mixed Gender and No Values</b>

In [5]:
#process genders and set labels
def value_in_borrower_gender(x):
    
    if type(x) is float and np.isnan(x):
        #assign label No Values
        return "No values"
    genders = x.split(",")
    male_count = sum(g.strip() == 'male' for g in genders)
    female_count = sum(g.strip() == 'female' for g in genders)
    
    if(male_count > 0 and female_count > 0):
        #assign label Mi
        return "Mixed Gender"
    elif(female_count > 0):
        return "Female"
    elif (male_count > 0):
        return "Male"


In [6]:
kiva_keras.borrower_genders = kiva_keras.borrower_genders.apply(value_in_borrower_gender)

In [7]:
#kiva_keras.borrower_genders.value_counts()

In [8]:
#process labels for description
#populate the columnn with the values Description and No Description if there is a text

kiva_keras.insert(loc = 3, column='description',value = 'Yes')
kiva_keras['use'].fillna('No', inplace=True)
condition_desc = (kiva_keras['use'] =='No')
kiva_keras.loc[condition_desc, 'description'] = 'No'
#kiva_keras.description.value_counts()

In [9]:
#drop the columns that are not necessary
kiva_keras.drop(['id', 'funded_amount','country_code','posted_time','disbursed_time','funded_time','tags','date','region','partner_id','use' ], axis=1, inplace=True)


In [10]:
#remove the three biggest loans
kiva_keras = kiva_keras.loc[kiva_keras['loan_amount'] < 50000]
#(Controllnan)kiva_keras.isnull().sum(axis = 0)


In [11]:
#save file wirh reduced columns for machine learning
kiva_keras.to_csv('data_files/kiva_keras.csv', index=False)


<b>Feature Scaling:</b>
Some of the variables have higher values (loan_amount)  while some have a lower range of values (term_in_months).If we don’t rescale the algorithm will give the loan_amount more predominance. 
<b>Label encoding:</b>In order to be able to compute all the features we need to turn categorical variables into a series of zeros and ones.

In [12]:

from sklearn.preprocessing import StandardScaler,MinMaxScaler


scaler = MinMaxScaler()


kiva_keras = pd.get_dummies(data=kiva_keras, columns=['activity','description', 'sector', 'country','currency','repayment_interval','borrower_genders'])
kiva_keras[['loan_amount','term_in_months','lender_count']] = scaler.fit_transform(kiva_keras[['loan_amount','term_in_months','lender_count']])



In [13]:
#Remove funded amount and separate th Y form the X
y = kiva_keras.loan_status
kiva_keras.drop(['loan_status'], axis=1, inplace=True)

In [14]:
#Separate Train and Test
from sklearn.model_selection import train_test_split
#creating test and train
X_train, X_test, y_train, y_test = train_test_split(kiva_keras, y, test_size=0.20,shuffle=True)

X_train_cv, X_cv, y_test_cv, y_cv = train_test_split(X_train,  y_train, test_size=0.30,shuffle=True)

<b>LabelEncoder</b>
Unfortunately machine learning algorithms can’t compute labels so we have to traduce the labels to numerical values.
<b>One hot encoding</b>
When we encode the labels each will get different numerical values. The first will become a 1 the second will become a 2 and so on. The problem with this method is the some former labels will have smaller values and some bigger. This introduces a rank for our labels that did not exist.
One hot encoding eliminates this rank by converting each encoded label into binary vector.


In [21]:
from keras.utils import to_categorical
from sklearn import preprocessing


# Encode train_y
le = preprocessing.LabelEncoder()
y_train_step_one = le.fit_transform(y_train)
keras_train_y = to_categorical(y_train_step_one)
# Encode train_y

y_test_step_one = le.fit_transform(y_test)
keras_test_y = to_categorical(y_test_step_one)

y_test_cv_step_one = le.fit_transform(y_cv)
keras_test_y_cv = to_categorical(y_test_cv_step_one)


<h2>Architecture:</h2>



<b>The Input Layer:</b>
The number of nodes is determined by the number of features

<b>The Outpulayer:</b> 
The number of nodes will be equal to the number of classes, wich in this case it will be three.

<b>The number of hidden layers:</b> 
The question to ask is whether we need one or more layers. According to The  universal approximation theorem single layer neural network can learn anything, when given appropriate parameters. So I will establish only one hidden layer.
Source: https://en.wikipedia.org/wiki/Universal_approximation_theorem

<b>The nodes of  hidden layer:</b> 
Deciding the number of nodes for the hidden layer is much more complex, and there is unfortunately no “Golden Rule” as the number will be influenced by the complexity of the problem, the amount of features, the size of the training set. 

<b>Activation functions:</b> 
I will use two different activation functions. The activation function for the output layer will be sigmoid as I am Interested in returning continuous values between 0 and 1 indicating the probability of each of the three outcomes.<br>
The other activation function which will be used for the first and for the hidden layer will be relu as it is more resistant to the vanishing gradient problem.

<b>Dropout</b> In order to reduce overfitting I will include Dropout,


<h2>Find the best parameters using Gridsearch</h2>


Rmsprop: is an  is a very robust optimizer that uses the moving average of squared gradients to normalize the gradient itself. It decrease the steps when it finds large gradients and increases the step for small gradient to avoid the vanishing gradient problem.

According to the Keras documentation it is recommended to leave most parameters untouched (default values). The only parameter to modify is the learning rate. I will change the parameter in two directions by increasing/decreasing the learning rate with +/- 0.0005.

Number of epochs: Is the number of times we sent our training data through the model. We want the number to be high enough to give the model the ability to learn.

Batch Size: The number of samples that we sent through the model. (one forward/backward pass.) Sending all the samples through the network would be computationally expensive, So instead of sending all the samples we start sending the first  n number (n being the batch size) of samples through the model, than the next n number of samples and so on.


In [16]:
#Building a basic model
import numpy as np
from keras.models import Sequential
from keras import layers
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_classification
from keras.optimizers import Adam,Adagrad,RMSprop,Nadam,SGD,Adadelta
from keras.layers import Dense,Dropout

# Function to create model, required for KerasClassifier
def create_model(optimizer):
    # create model
    model = Sequential()
    model.add(Dense(10, input_shape=kiva_keras.shape[1:], activation='relu'))
    model.add(Dense(50,  activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(3, activation='sigmoid'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model


In [17]:
# create model
model = KerasClassifier(build_fn=create_model, verbose=0)
# grid search epochs, batch size and optimizer
optimizers = ['rmsprop', 'SGD', 'Adagrad']
epochs = [40, 80]
batches = [1000,2000]
param_grid = dict(optimizer=optimizers, epochs=epochs, batch_size=batches)
grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result = grid.fit(X_cv,keras_test_y_cv,validation_split = 0.2,verbose=1)


Train on 85908 samples, validate on 21478 samples
Epoch 1/40
85908/85908 [==============================] - 4s 41us/step - loss: 0.6600 - acc: 0.9122 - val_loss: 0.2877 - val_acc: 0.9314
Epoch 2/40
85908/85908 [==============================] - 3s 37us/step - loss: 0.2722 - acc: 0.9278 - val_loss: 0.2371 - val_acc: 0.9314
Epoch 3/40
85908/85908 [==============================] - 3s 38us/step - loss: 0.2472 - acc: 0.9278 - val_loss: 0.2304 - val_acc: 0.9314
Epoch 4/40
85908/85908 [==============================] - 4s 52us/step - loss: 0.2411 - acc: 0.9278 - val_loss: 0.2270 - val_acc: 0.9314
Epoch 5/40
85908/85908 [==============================] - 4s 49us/step - loss: 0.2379 - acc: 0.9279 - val_loss: 0.2254 - val_acc: 0.9314
Epoch 6/40
85908/85908 [==============================] - 4s 46us/step - loss: 0.2350 - acc: 0.9280 - val_loss: 0.2238 - val_acc: 0.9315
Epoch 7/40
85908/85908 [==============================] - 3s 37us/step - loss: 0.2329 - acc: 0.9283 - val_loss: 0.2223 - val_acc

Epoch 20/40
85908/85908 [==============================] - 3s 35us/step - loss: 0.2080 - acc: 0.9317 - val_loss: 0.1986 - val_acc: 0.9351
Epoch 21/40
85908/85908 [==============================] - 3s 34us/step - loss: 0.2054 - acc: 0.9328 - val_loss: 0.1965 - val_acc: 0.9367
Epoch 22/40
85908/85908 [==============================] - 3s 34us/step - loss: 0.2032 - acc: 0.9334 - val_loss: 0.1932 - val_acc: 0.9371
Epoch 23/40
85908/85908 [==============================] - 3s 34us/step - loss: 0.2002 - acc: 0.9348 - val_loss: 0.1914 - val_acc: 0.9375
Epoch 24/40
85908/85908 [==============================] - 3s 34us/step - loss: 0.1979 - acc: 0.9353 - val_loss: 0.1886 - val_acc: 0.9387
Epoch 25/40
85908/85908 [==============================] - 3s 35us/step - loss: 0.1950 - acc: 0.9361 - val_loss: 0.1883 - val_acc: 0.9409
Epoch 26/40
85908/85908 [==============================] - 3s 34us/step - loss: 0.1924 - acc: 0.9376 - val_loss: 0.1841 - val_acc: 0.9400
Epoch 27/40
85908/85908 [=========

85908/85908 [==============================] - 3s 34us/step - loss: 0.1601 - acc: 0.9475 - val_loss: 0.1669 - val_acc: 0.9474
Epoch 40/40
85908/85908 [==============================] - 3s 33us/step - loss: 0.1587 - acc: 0.9476 - val_loss: 0.1674 - val_acc: 0.9484
Train on 85908 samples, validate on 21478 samples
Epoch 1/40
85908/85908 [==============================] - 3s 38us/step - loss: 1.0021 - acc: 0.8143 - val_loss: 0.8925 - val_acc: 0.9314
Epoch 2/40
85908/85908 [==============================] - 3s 37us/step - loss: 0.7890 - acc: 0.9278 - val_loss: 0.6690 - val_acc: 0.9314
Epoch 3/40
85908/85908 [==============================] - 3s 34us/step - loss: 0.5631 - acc: 0.9278 - val_loss: 0.4411 - val_acc: 0.9314
Epoch 4/40
85908/85908 [==============================] - 3s 34us/step - loss: 0.3931 - acc: 0.9278 - val_loss: 0.3231 - val_acc: 0.9314
Epoch 5/40
85908/85908 [==============================] - 3s 34us/step - loss: 0.3269 - acc: 0.9278 - val_loss: 0.2871 - val_acc: 0.9314
E

Epoch 18/40
85908/85908 [==============================] - 3s 33us/step - loss: 0.2766 - acc: 0.9267 - val_loss: 0.2553 - val_acc: 0.9314
Epoch 19/40
85908/85908 [==============================] - 3s 34us/step - loss: 0.2756 - acc: 0.9267 - val_loss: 0.2548 - val_acc: 0.9314
Epoch 20/40
85908/85908 [==============================] - 3s 34us/step - loss: 0.2754 - acc: 0.9267 - val_loss: 0.2543 - val_acc: 0.9314
Epoch 21/40
85908/85908 [==============================] - 3s 34us/step - loss: 0.2741 - acc: 0.9267 - val_loss: 0.2539 - val_acc: 0.9314
Epoch 22/40
85908/85908 [==============================] - 3s 34us/step - loss: 0.2730 - acc: 0.9267 - val_loss: 0.2535 - val_acc: 0.9314
Epoch 23/40
85908/85908 [==============================] - 3s 33us/step - loss: 0.2731 - acc: 0.9267 - val_loss: 0.2531 - val_acc: 0.9314
Epoch 24/40
85908/85908 [==============================] - 3s 34us/step - loss: 0.2723 - acc: 0.9267 - val_loss: 0.2527 - val_acc: 0.9314
Epoch 25/40
85908/85908 [=========

85908/85908 [==============================] - 4s 44us/step - loss: 0.2676 - acc: 0.9276 - val_loss: 0.2535 - val_acc: 0.9289
Epoch 38/40
85908/85908 [==============================] - 3s 35us/step - loss: 0.2673 - acc: 0.9276 - val_loss: 0.2533 - val_acc: 0.9289
Epoch 39/40
85908/85908 [==============================] - 3s 35us/step - loss: 0.2671 - acc: 0.9276 - val_loss: 0.2531 - val_acc: 0.9289
Epoch 40/40
85908/85908 [==============================] - 3s 36us/step - loss: 0.2663 - acc: 0.9276 - val_loss: 0.2529 - val_acc: 0.9289
Train on 85908 samples, validate on 21478 samples
Epoch 1/40
85908/85908 [==============================] - 4s 42us/step - loss: 0.3399 - acc: 0.9254 - val_loss: 0.2440 - val_acc: 0.9314
Epoch 2/40
85908/85908 [==============================] - 3s 35us/step - loss: 0.2541 - acc: 0.9278 - val_loss: 0.2345 - val_acc: 0.9314
Epoch 3/40
85908/85908 [==============================] - 3s 35us/step - loss: 0.2468 - acc: 0.9278 - val_loss: 0.2306 - val_acc: 0.9314

...

85908/85908 [==============================] - 3s 33us/step - loss: 0.3162 - acc: 0.9267 - val_loss: 0.2820 - val_acc: 0.9314
Epoch 12/80
85908/85908 [==============================] - 3s 32us/step - loss: 0.3065 - acc: 0.9267 - val_loss: 0.2747 - val_acc: 0.9314
Epoch 13/80
85908/85908 [==============================] - 3s 33us/step - loss: 0.2998 - acc: 0.9267 - val_loss: 0.2701 - val_acc: 0.9314
Epoch 14/80
85908/85908 [==============================] - 3s 33us/step - loss: 0.2958 - acc: 0.9267 - val_loss: 0.2670 - val_acc: 0.9314
Epoch 15/80
85908/85908 [==============================] - 3s 33us/step - loss: 0.2935 - acc: 0.9267 - val_loss: 0.2648 - val_acc: 0.9314
Epoch 16/80
85908/85908 [==============================] - 3s 33us/step - loss: 0.2907 - acc: 0.9267 - val_loss: 0.2631 - val_acc: 0.9314
Epoch 17/80
85908/85908 [==============================] - 3s 33us/step - loss: 0.2885 - acc: 0.9267 - val_loss: 0.2618 - val_acc: 0.9314
Epoch 18/80
85908/85908 [=====================

85908/85908 [==============================] - 3s 33us/step - loss: 0.2668 - acc: 0.9276 - val_loss: 0.2553 - val_acc: 0.9289
Epoch 50/80
85908/85908 [==============================] - 3s 32us/step - loss: 0.2675 - acc: 0.9276 - val_loss: 0.2552 - val_acc: 0.9289
Epoch 51/80
85908/85908 [==============================] - 3s 33us/step - loss: 0.2674 - acc: 0.9276 - val_loss: 0.2550 - val_acc: 0.9289
Epoch 52/80
85908/85908 [==============================] - 3s 33us/step - loss: 0.2667 - acc: 0.9276 - val_loss: 0.2549 - val_acc: 0.9289
Epoch 53/80
85908/85908 [==============================] - 3s 34us/step - loss: 0.2665 - acc: 0.9276 - val_loss: 0.2548 - val_acc: 0.9289
Epoch 54/80
85908/85908 [==============================] - 4s 48us/step - loss: 0.2663 - acc: 0.9276 - val_loss: 0.2546 - val_acc: 0.9289
Epoch 55/80
85908/85908 [==============================] - 3s 37us/step - loss: 0.2662 - acc: 0.9276 - val_loss: 0.2545 - val_acc: 0.9289
Epoch 56/80
85908/85908 [=====================

85908/85908 [==============================] - 3s 33us/step - loss: 0.2440 - acc: 0.9267 - val_loss: 0.2282 - val_acc: 0.9314
Epoch 8/80
85908/85908 [==============================] - 3s 32us/step - loss: 0.2430 - acc: 0.9268 - val_loss: 0.2272 - val_acc: 0.9314
Epoch 9/80
85908/85908 [==============================] - 3s 33us/step - loss: 0.2406 - acc: 0.9270 - val_loss: 0.2264 - val_acc: 0.9317
Epoch 10/80
85908/85908 [==============================] - 3s 32us/step - loss: 0.2399 - acc: 0.9272 - val_loss: 0.2257 - val_acc: 0.9320
Epoch 11/80
85908/85908 [==============================] - 3s 33us/step - loss: 0.2376 - acc: 0.9275 - val_loss: 0.2249 - val_acc: 0.9323
Epoch 12/80
85908/85908 [==============================] - 3s 31us/step - loss: 0.2370 - acc: 0.9276 - val_loss: 0.2242 - val_acc: 0.9325
Epoch 13/80
85908/85908 [==============================] - 3s 33us/step - loss: 0.2363 - acc: 0.9277 - val_loss: 0.2235 - val_acc: 0.9327
Epoch 14/80
85908/85908 [=======================

85908/85908 [==============================] - 3s 32us/step - loss: 0.2231 - acc: 0.9276 - val_loss: 0.2210 - val_acc: 0.9289
Epoch 46/80
85908/85908 [==============================] - 3s 33us/step - loss: 0.2226 - acc: 0.9275 - val_loss: 0.2208 - val_acc: 0.9289
Epoch 47/80
85908/85908 [==============================] - 3s 33us/step - loss: 0.2225 - acc: 0.9276 - val_loss: 0.2205 - val_acc: 0.9289
Epoch 48/80
85908/85908 [==============================] - 3s 33us/step - loss: 0.2225 - acc: 0.9276 - val_loss: 0.2203 - val_acc: 0.9289
Epoch 49/80
85908/85908 [==============================] - 3s 33us/step - loss: 0.2212 - acc: 0.9276 - val_loss: 0.2200 - val_acc: 0.9289
Epoch 50/80
85908/85908 [==============================] - 3s 33us/step - loss: 0.2215 - acc: 0.9276 - val_loss: 0.2198 - val_acc: 0.9289
Epoch 51/80
85908/85908 [==============================] - 3s 33us/step - loss: 0.2212 - acc: 0.9276 - val_loss: 0.2195 - val_acc: 0.9289
Epoch 52/80
85908/85908 [=====================

Epoch 24/80
128863/128863 [==============================] - 5s 38us/step - loss: 0.1657 - acc: 0.9455 - val_loss: 0.1621 - val_acc: 0.9456
Epoch 25/80
128863/128863 [==============================] - 5s 38us/step - loss: 0.1631 - acc: 0.9464 - val_loss: 0.1609 - val_acc: 0.9471
Epoch 26/80
128863/128863 [==============================] - 5s 37us/step - loss: 0.1607 - acc: 0.9471 - val_loss: 0.1595 - val_acc: 0.9471
Epoch 27/80
128863/128863 [==============================] - 5s 38us/step - loss: 0.1592 - acc: 0.9477 - val_loss: 0.1556 - val_acc: 0.9476
Epoch 28/80
128863/128863 [==============================] - 5s 37us/step - loss: 0.1574 - acc: 0.9484 - val_loss: 0.1536 - val_acc: 0.9495
Epoch 29/80
128863/128863 [==============================] - 5s 38us/step - loss: 0.1557 - acc: 0.9487 - val_loss: 0.1511 - val_acc: 0.9498
Epoch 30/80
128863/128863 [==============================] - 5s 38us/step - loss: 0.1550 - acc: 0.9491 - val_loss: 0.1484 - val_acc: 0.9500
Epoch 31/80
128863/1

In [18]:
print(grid_result.best_params_,grid_result.best_score_)

{'batch_size': 1000, 'epochs': 80, 'optimizer': 'rmsprop'} 0.9524705259263068


The Gridsearch gave the following best paramters: Batch Size =  1000, Epochs: 80 and Rmsprop as an optimizer. So I am going to implement the Neural Network to see how it performs. I will also add a Checkpointer in order to get and save best wights

In [19]:
import numpy as np
from keras.models import Sequential
from keras import layers
from keras.optimizers import RMSprop
from keras.layers import Dense,Dropout
from keras.callbacks import ModelCheckpoint  

model = Sequential()
model.add(Dense(10, input_shape=kiva_keras.shape[1:], activation='relu'))
model.add(Dense(75,  activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(3, activation='sigmoid'))
  
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath='weights.besto.hdf5', 
                               verbose=1, save_best_only=True)

model.fit(X_train,keras_train_y,validation_split = 0.2,epochs = 80,batch_size= 1000,callbacks=[checkpointer],verbose=1)


Train on 429543 samples, validate on 107386 samples
Epoch 1/80
429543/429543 [==============================] - 22s 51us/step - loss: 0.3066 - acc: 0.9257 - val_loss: 0.2335 - val_acc: 0.9266

Epoch 00001: val_loss improved from inf to 0.23346, saving model to weights.besto.hdf5
Epoch 2/80
429543/429543 [==============================] - 20s 47us/step - loss: 0.2303 - acc: 0.9283 - val_loss: 0.2257 - val_acc: 0.9277

Epoch 00002: val_loss improved from 0.23346 to 0.22568, saving model to weights.besto.hdf5
Epoch 3/80
429543/429543 [==============================] - 20s 47us/step - loss: 0.2216 - acc: 0.9296 - val_loss: 0.2159 - val_acc: 0.9288

Epoch 00003: val_loss improved from 0.22568 to 0.21595, saving model to weights.besto.hdf5
Epoch 4/80
429543/429543 [==============================] - 20s 47us/step - loss: 0.2110 - acc: 0.9320 - val_loss: 0.2058 - val_acc: 0.9329

Epoch 00004: val_loss improved from 0.21595 to 0.20584, saving model to weights.besto.hdf5
Epoch 5/80
429543/429543

429543/429543 [==============================] - 20s 46us/step - loss: 0.1134 - acc: 0.9592 - val_loss: 0.1159 - val_acc: 0.9582

Epoch 00078: val_loss did not improve from 0.11336
Epoch 79/80
429543/429543 [==============================] - 20s 47us/step - loss: 0.1131 - acc: 0.9592 - val_loss: 0.1135 - val_acc: 0.9589

Epoch 00079: val_loss did not improve from 0.11336
Epoch 80/80
429543/429543 [==============================] - 20s 47us/step - loss: 0.1128 - acc: 0.9593 - val_loss: 0.1139 - val_acc: 0.9582

Epoch 00080: val_loss did not improve from 0.11336


In [22]:


model = Sequential()
model.add(Dense(10, input_shape=kiva_keras.shape[1:], activation='relu'))
model.add(Dense(75,  activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(3, activation='sigmoid'))

model.load_weights("weights.besto.hdf5")
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])
model.fit(X_train,keras_train_y,batch_size= 1000)
score = model.evaluate(X_test,keras_test_y)


Epoch 1/1
134233/134233 [==============================] - 12s 88us/step


In [23]:
print(score)

[0.12888269318409254, 0.9525079525898996]


In [24]:
from sklearn.metrics import classification_report
import numpy as np

keras_test_y = np.argmax(keras_test_y, axis=1) # Convert one-hot to index
y_pred = model.predict_classes(X_test)
print(classification_report(keras_test_y, y_pred, target_names=['full funded', 'non funded','partially funded']))

                  precision    recall  f1-score   support

     full funded       0.96      0.99      0.98    124615
      non funded       0.71      0.79      0.75       682
partially funded       0.78      0.43      0.56      8936

     avg / total       0.95      0.95      0.95    134233



<h2>Finetuning the architecture by manual approach</h2>

 I will focus on one element of the model and one parameter of the optimizer. 
 
The first thing I will change is the number of nodes of the hidden layer. I searched out both Stackoverflow and Stackexchange to get more information on how to find the ideal number of nodes and there is unfortunately no golden rule. Besides that the number of nodes is influenced by the complexity of the oroblem, the number of samples and the number of nodes from both the input and output layers.
 
 So our range of nodes for the hidden layer can be between a couple of nodes and thousands of nodes. The proplem with increasing the number of nodes too much is, that we maybe can create a modell that is able to learn the patterns in our data and predict fine when presented to known data but fails when presented to unseen data.
<br>
I increased the number of nodes gradually and increased the percentage of dropout to reduce overfitting.
<br>
According to the keras dokumention the only paramter that is suitable for optimizing is the learning rate. I will try two directions. I will take the default 0.001 and add/substract 0.0005 


    
    


In [25]:
model = Sequential()
model.add(Dense(10, input_shape=kiva_keras.shape[1:], activation='relu'))
model.add(Dense(1000,  activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='sigmoid'))
  
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.0015), metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath='weights.bestopt.hdf5', 
                               verbose=1, save_best_only=True)

model.fit(X_train,keras_train_y,validation_split = 0.2,epochs = 80,batch_size= 1000,callbacks=[checkpointer],verbose=1)



Train on 429543 samples, validate on 107386 samples
Epoch 1/80
429543/429543 [==============================] - 61s 142us/step - loss: 0.2868 - acc: 0.9265 - val_loss: 0.2379 - val_acc: 0.9266

Epoch 00001: val_loss improved from inf to 0.23789, saving model to weights.bestopt.hdf5
Epoch 2/80
429543/429543 [==============================] - 58s 135us/step - loss: 0.2295 - acc: 0.9286 - val_loss: 0.2301 - val_acc: 0.9275

Epoch 00002: val_loss improved from 0.23789 to 0.23011, saving model to weights.bestopt.hdf5
Epoch 3/80
429543/429543 [==============================] - 55s 127us/step - loss: 0.2218 - acc: 0.9290 - val_loss: 0.2207 - val_acc: 0.9280

Epoch 00003: val_loss improved from 0.23011 to 0.22071, saving model to weights.bestopt.hdf5
Epoch 4/80
429543/429543 [==============================] - 58s 135us/step - loss: 0.2123 - acc: 0.9301 - val_loss: 0.2094 - val_acc: 0.9301

Epoch 00004: val_loss improved from 0.22071 to 0.20937, saving model to weights.bestopt.hdf5
Epoch 5/80
4

429543/429543 [==============================] - 55s 129us/step - loss: 0.1237 - acc: 0.9561 - val_loss: 0.1255 - val_acc: 0.9545

Epoch 00039: val_loss improved from 0.12560 to 0.12551, saving model to weights.bestopt.hdf5
Epoch 40/80
429543/429543 [==============================] - 54s 125us/step - loss: 0.1231 - acc: 0.9561 - val_loss: 0.1298 - val_acc: 0.9539

Epoch 00040: val_loss did not improve from 0.12551
Epoch 41/80
429543/429543 [==============================] - 54s 125us/step - loss: 0.1229 - acc: 0.9560 - val_loss: 0.1254 - val_acc: 0.9547

Epoch 00041: val_loss improved from 0.12551 to 0.12542, saving model to weights.bestopt.hdf5
Epoch 42/80
429543/429543 [==============================] - 53s 123us/step - loss: 0.1222 - acc: 0.9564 - val_loss: 0.1338 - val_acc: 0.9535

Epoch 00042: val_loss did not improve from 0.12542
Epoch 43/80
429543/429543 [==============================] - 53s 124us/step - loss: 0.1221 - acc: 0.9562 - val_loss: 0.1260 - val_acc: 0.9547

Epoch 000

In [1]:
from sklearn.metrics import classification_report
import numpy as np
model.load_weights("weights.bestopt.hdf5")
keras_test_y = np.argmax(keras_test_y, axis=1) # Convert one-hot to index
y_pred = model.predict_classes(X_test)
print(classification_report(keras_test_y, y_pred, target_names=['full funded', 'non funded','partially funded']))

NameError: name 'model' is not defined

Comparison Sklearn and Keras 

Sklearn REnadom Forest achieved godd result in a deccent amount of timer, Sklearn was much better at predicitng the smallest class but performed worse for the partially funded loans. Keras had more provblems identifying the smallest class and had a better performance for the partially fundet loans. Note that the Skleatrn MLP showed the same patterns regarding the classes. Sklearn was easy to implement while a neaural network requires a bigger effort. 

Based on the consecuences of correct/uncorrect classifications I would choose the Randomforest based on the assumption that getting no funding at all is a worse outcome than getting some funding. So it's more critical to classify correctly the non fundet loan. 